In [4]:
import pandas as pd
import numpy as np

In [5]:
# import data sets

blooms_raw = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQms0ohdtDlTJoP9tJsurwp1axTMJJxHO2--j0qF2m9moLasFUUbwkQr2XApGqGcA/pub?output=csv')
temps_raw = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTe2xv8DuQALVpmmzeXraIFFXd4rv4jSefxsAy2OvapY2Zv91RazrZfISSczhyyHA/pub?output=csv')

In [6]:
# finding common column name for merge

print(blooms_raw.columns)
print(temps_raw.columns)

Index(['Year', 'Doy', 'Calendar_date'], dtype='object')
Index(['Year', '3_yr_temp_avg', '16_yr_temp_avg', '22_yr_temp_avg', 'obs_temp',
       'urban_effect', 'upper_ci', 'lower_ci'],
      dtype='object')


In [7]:
# merge datasets on year

df = blooms_raw.merge(temps_raw,
                            how='left',
                            left_on='Year',
                            right_on='Year')

# preliminary count of rows and columns

df.shape

(1217, 10)

In [63]:
# check for missing  values

count_df = df.isna().sum().sum()
print('Count of missing values: ' + str(count_df))

# number of missing values by column

df.isna().sum()

Count of missing values: 4575


Year                 0
Doy                432
Calendar_date      432
3_yr_temp_avg      131
16_yr_temp_avg     436
22_yr_temp_avg     625
obs_temp          1122
urban_effect      1135
upper_ci           131
lower_ci           131
dtype: int64

In [8]:
# drop rows with missing values in calendar_date and doy
# drop 16 year and 22 yr average due to high number of missing values
# confirm some rows were dropped

df = df.dropna(subset=['Doy', 'Calendar_date', '3_yr_temp_avg'])
df = df.drop(columns=['16_yr_temp_avg', '22_yr_temp_avg', 'obs_temp'])
count_dfs = df.isna().sum().sum()
print('Count of missing values: ' + str(count_dfs))
df.shape

# confirmation of cleaned rows and columns
df.isna().sum()

Count of missing values: 681


Year               0
Doy                0
Calendar_date      0
3_yr_temp_avg      0
urban_effect     681
upper_ci           0
lower_ci           0
dtype: int64

In [9]:
# convert from celsius to fahrenheit
# drop celsuis columns

df = df.assign(temp_fahrenheit = lambda x: (9/5)*x['3_yr_temp_avg']+32, upper_ci_f =lambda x: (9/5)*x['upper_ci']+32, lower_ci_f =lambda x: (9/5)*x['lower_ci']+32)
df = df.drop(columns=['3_yr_temp_avg', 'upper_ci', 'lower_ci'])

# confirmation that celsius columns were dropped and fahrenheit columns were added
df.head()

,Year,Doy,Calendar_date,urban_effect,temp_fahrenheit,upper_ci_f,lower_ci_f
52,853,104.0,14-Apr,NaN,42.512,49.964,35.060
63,864,100.0,9-Apr,NaN,44.834,52.358,37.310
65,866,106.0,16-Apr,NaN,45.284,53.780,36.788
90,891,109.0,19-Apr,NaN,42.080,44.690,39.470
91,892,108.0,17-Apr,NaN,42.278,44.600,39.956


In [10]:
# finding quantiles
df.temp_fahrenheit.quantile([0.25,0.5, 0.75])

0.25    42.044
0.50    42.890
0.75    43.520
Name: temp_fahrenheit, dtype: float64

In [11]:
# recode temperature into categories for temp: high, middle, low
# high = top 25%
# medium = middle 50%
# low = bottom 25%

df = (df.assign(temp_cat = lambda x: pd.cut(x['temp_fahrenheit'],
bins=[0, 42.045, 42.891, 100],
labels=["Low", "Medium", "High"])))
df.head()

,Year,Doy,Calendar_date,urban_effect,temp_fahrenheit,upper_ci_f,lower_ci_f,temp_cat
52,853,104.0,14-Apr,NaN,42.512,49.964,35.060,Medium
63,864,100.0,9-Apr,NaN,44.834,52.358,37.310,High
65,866,106.0,16-Apr,NaN,45.284,53.780,36.788,High
90,891,109.0,19-Apr,NaN,42.080,44.690,39.470,Medium
91,892,108.0,17-Apr,NaN,42.278,44.600,39.956,Medium


In [13]:
# export final dataset
df.to_excel(r'C:/Users/britt/Documents/Python/sakura/sakura_blooms_kyoto_japan/sakura.xlsx', index = False)